# Random forests imputation

Impute a random realization of a random forests model using CPS data.

For this example, use Schedule C (`e00900`), since it can be positive, zero, or negative, like Schedule E in https://github.com/open-source-economics/taxdata/issues/221.

## Setup

### Imports

In [1]:
import taxcalc as tc
import pandas as pd
import numpy as np
from sklearn import ensemble
from sklearn import linear_model
from sklearn import metrics
from sklearn import model_selection
import os

In [2]:
tc.__version__

'0.20.1'

## Data

Get raw CPS records.

In [3]:
data = os.path.join(tc.Records.CUR_PATH, 'cps.csv.gz')
df = pd.read_csv(data)

Remove `e00900p` and `e00900s`--the taxpayer and spouse Schedule C components--and describe the data.

In [4]:
df.drop(['e00900p', 'e00900s'], axis=1, inplace=True)

In [5]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
age_head,456465.0,49.614030,1.715312e+01,0.0,36.0,50.0,63.0,8.500000e+01
age_spouse,456465.0,27.416347,2.770706e+01,0.0,0.0,30.0,52.0,8.500000e+01
e00200p,456465.0,47805.744890,1.266239e+05,0.0,0.0,13511.0,50764.0,4.339028e+06
e02100p,456465.0,15337.615933,1.777691e+05,-19998.0,0.0,0.0,0.0,1.157928e+07
e00200s,456465.0,26644.054000,8.937828e+04,0.0,0.0,0.0,20537.0,4.526829e+06
e02100s,456465.0,6296.968872,1.052554e+05,-19998.0,0.0,0.0,0.0,9.677610e+06
a_lineno,456465.0,1.370892,9.262111e-01,1.0,1.0,1.0,1.0,1.600000e+01
e00600,456465.0,8724.914673,1.594430e+06,0.0,0.0,0.0,73.0,1.075466e+09
e00800,456465.0,130.100146,1.497370e+04,0.0,0.0,0.0,0.0,5.440465e+06
e01500,456465.0,11824.040174,3.894361e+04,0.0,0.0,0.0,0.0,1.447936e+06


## Model

Train a random forests model.

In [6]:
YCOL = 'e00900'

In [7]:
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(
    df.drop(YCOL, axis=1), df[YCOL], random_state=3)

In [8]:
Y_train_sign = np.sign(Y_train)
Y_test_sign = np.sign(Y_test)

In [9]:
# Reduce for faster runtime.
N_ESTIMATORS = 100
rf = ensemble.RandomForestRegressor(n_estimators=N_ESTIMATORS, 
                                    min_samples_leaf=1, random_state=3, 
                                    verbose=True, 
                                    n_jobs=-1)  # Use maximum number of cores.
rf.fit(X_train, Y_train)

[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  7.2min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 16.7min finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
           oob_score=False, random_state=3, verbose=True, warm_start=False)

### Model description

Show the ten most important features.

In [10]:
feature_importance = pd.Series(rf.feature_importances_, index=X_train.columns)
feature_importance.sort_values(ascending=False)[:10]

e00300     0.086969
e00650     0.070918
e18400     0.048008
e00200s    0.045677
e20400     0.042812
h_seq      0.041276
e18500     0.038607
RECID      0.036186
e02100     0.035376
e03300     0.034689
dtype: float64

## Predict

### Top-line (average)

In [11]:
pred = pd.DataFrame({'actual': Y_test,
                     'pred': rf.predict(X_test)})
pred['error'] = pred.pred - pred.actual
pred['actual_sign'] = np.sign(pred.actual)
pred['pred_sign'] = np.sign(pred.pred)
pred['correct_sign'] = (pred.actual_sign == pred.pred_sign)
pred['count'] = 1

[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    1.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    2.5s finished


MAE, RMSE, and % negative/zero/positive.

In [12]:
pred.error.abs().mean()

5282.16166828781

In [13]:
pred.error.pow(2).mean() ** 0.5

63886.85751757012

In [14]:
pred.pivot_table(index='actual_sign', columns='pred_sign', values='count', 
                 aggfunc=sum, margins=True)

pred_sign,-1.0,0.0,1.0,All
actual_sign,,,,
-1,155,28,212,395
0,2487,51180,43159,96826
1,71,188,16637,16896
All,2713,51396,60008,114117


In [15]:
pred.correct_sign.mean()

0.5956343051429673

### All trees

In [16]:
preds = []
for estimator in rf.estimators_:
    preds.append(estimator.predict(X_test))
preds = np.array(preds).transpose()  # One row per record.

In [17]:
rand_col = np.random.randint(N_ESTIMATORS, size=preds.shape[0])
random_tree = preds[np.arange(preds.shape[0]), rand_col.ravel()]

In [18]:
pred_random_tree = pd.DataFrame({'actual': Y_test,
                                 'pred': random_tree})
pred_random_tree['error'] = pred_random_tree.pred - pred_random_tree.actual
pred_random_tree['actual_sign'] = np.sign(pred_random_tree.actual)
pred_random_tree['pred_sign'] = np.sign(pred_random_tree.pred)
pred_random_tree['correct_sign'] = (
    pred_random_tree.actual_sign == pred_random_tree.pred_sign)
pred_random_tree['count'] = 1

As expected, MAE and RMSE exceed values from the point estimate prediction.

In [19]:
pred_random_tree.error.abs().mean()

6874.652339265841

In [20]:
pred_random_tree.error.pow(2).mean() ** 0.5

120550.85142417901

But the distribution of sign is closer to correct, since it's not averaging out the zeros.

In [21]:
pred_random_tree.pivot_table(index='actual_sign', columns='pred_sign', 
                             values='count', aggfunc=sum, margins=True)

pred_sign,-1.0,0.0,1.0,All
actual_sign,,,,
-1,136,208,51,395
0,193,93417,3216,96826
1,48,2724,14124,16896
All,377,96349,17391,114117


In [22]:
pred_random_tree.correct_sign.mean()

0.9435666903265947

#### Log-loss of sign

In [23]:
preds_neg = np.sum(preds < 0, axis=1) / 100
preds_zero = np.sum(preds == 0, axis=1) / 100
preds_pos = np.sum(preds > 0, axis=1) / 100

rf_pred_proba = list(map(list, zip(*[preds_neg, preds_zero, preds_pos])))

metrics.log_loss(Y_test_sign, rf_pred_proba)

0.17439860118741296

## Multinomial logistic regression of sign

In [24]:
mult = linear_model.LogisticRegression(
    multi_class='multinomial', solver='newton-cg', random_state=3)
mult.fit(X_train, Y_train_sign)

/home/mghenis/miniconda3/lib/python3.6/site-packages/sklearn/utils/optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=3, solver='newton-cg',
          tol=0.0001, verbose=0, warm_start=False)

### Coefficients

p-values are tricky to calculate, so just show the features with highest absolute coefficient on the zero class.

In [25]:
mult_coef = pd.DataFrame({
    'negative': mult.coef_[0],
    'zero': mult.coef_[1],
    'positive': mult.coef_[2],
    'abs_zero': np.abs(mult.coef_[1])},
    index=X_train.columns)

mult_coef.sort_values('abs_zero', ascending=False)[:10]

,abs_zero,negative,positive,zero
age_spouse,0.008752,0.000489,0.008263,-0.008752
agi_bin,0.002372,-0.000003,0.002375,-0.002372
fips,0.001438,0.000031,-0.001469,0.001438
age_head,0.001303,0.000071,0.001232,-0.001303
s006,0.001141,0.000710,-0.001851,0.001141
FLPDYR,0.001078,-0.001672,0.000594,0.001078
XTOT,0.000445,0.000008,0.000437,-0.000445
a_lineno,0.000359,-0.000015,-0.000344,0.000359
n21,0.000314,0.000012,0.000302,-0.000314
MARS,0.000200,0.000005,0.000195,-0.000200


### Predict

In [26]:
mult_pred = pd.DataFrame({'actual': Y_test_sign,
                          'pred': mult.predict(X_test)})
mult_pred['sign_correct'] = (mult_pred.actual == mult_pred.pred)
mult_pred['count'] = 1
mult_pred.sign_correct.mean()

0.8783178667507908

In [27]:
mult_pred.pivot_table(index='actual', columns='pred', 
                      values='count', aggfunc=sum, margins=True)

pred,-1,0,1,All
actual,,,,
-1,NaN,364.0,31.0,395
0,1.0,95474.0,1351.0,96826
1,1.0,12138.0,4757.0,16896
All,2.0,107976.0,6139.0,114117


#### Log-loss

In [28]:
mult_pred_proba = mult.predict_proba(X_test)
metrics.log_loss(Y_test_sign, mult_pred_proba)

0.3514170522996822